In [ ]:
# I have trained the Resnet Model from scratch
# I also loaded the best model I have on the resnet18.pt file, which goes up to 96.24% on test data and 100% on training data

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
input_size = (224,224)
batch_size = 16
#from __future__ import print_function, division
#cudnn.benchmark = True
plt.ion()   # interactive mode
criterion = nn.CrossEntropyLoss()
# Data Transformations

data_dir = '../input/vgg-data/z'
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class_names = image_datasets['train'].classes



In [ ]:
epsilon_vals = [.05,.1,.15,.2,.25,.3]

In [ ]:
!pip install torchattacks

In [ ]:
num_classes = 18
import torch
from torchvision import transforms
#from efficientnet_pytorch import EfficientNet
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, num_classes)

In [ ]:
import torchattacks
from torch.optim import lr_scheduler

model_ft.load_state_dict(torch.load('../input/resnet18/resnet18.pt'))
model_ft = model_ft.to("cuda")
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
 
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=14, gamma=0.1)

In [ ]:
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Step 4: Train the ART classifier

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=3)

# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

In [ ]:
train_loss_history = []
val_acc_history = []
val_loss_history = []
train_acc_history = []


In [ ]:
print(train_loss_history)
print(train_acc_history)


In [ ]:
def plot_it(nums):
    plt.figure(figsize=(25,20))
    f, ax = plt.subplots(1)
    ax.set_ylim(.9, 1)
    plt.plot(nums)


In [ ]:
val_loss = []
val_acc = []

In [ ]:
print(val_loss)

In [ ]:
print(val_acc)

In [ ]:
plt.plot(epsilon_vals,val_acc)

In [ ]:
plt.plot(epsilon_vals,val_loss)

In [ ]:
print(val)

In [ ]:
def test_model(model, criterion, attack):
    model.eval()  
    running_loss = 0
    running_corrects = 0
    for inputs, labels in dataloaders_dict['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        adv_images = attack(inputs, labels)
        outputs = model(adv_images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        for x in range(len(labels)):
            if labels[x]==preds[x]:
                running_corrects+=1
        #running_corrects += torch.sum(preds == labels.data)
        
    #print(running_corrects)
                 
    loss = running_loss / len(image_datasets['val'])
    acc = running_corrects / len(image_datasets['val'])
    print("Loss: ", loss)
    print("Acc: ",acc)
    return loss, acc
    

In [ ]:
val_loss = []
val_acc = []
for y in range(len(epsilon_vals)):
    loss = 0
    acc = 0
    criterion = nn.CrossEntropyLoss()
    attack = torchattacks.FGSM(model_ft, eps = epsilon_vals[y])
    loss, acc = test_model(model_ft, criterion, attack)
    val_loss.append(loss)
    val_acc.append(acc)


In [ ]:
plt.plot(epsilon_vals, val_acc)

In [ ]:
plt.plot(epsilon_vals, val_loss)

In [ ]:
val_loss = []
val_acc = []

for y in range(len(epsilon_vals)):
    loss = 0
    acc = 0
    criterion = nn.CrossEntropyLoss()
    attack = torchattacks.BIM(model_ft, eps=epsilon_vals[y], alpha=1/255, steps=0)
    loss, acc = test_model(model_ft, criterion, attack)
    val_loss.append(loss)
    val_acc.append(acc)



In [ ]:
plt.plot(epsilon_vals,val_acc)

In [ ]:
plt.plot(epsilon_vals, val_loss)

In [ ]:
c_vals = [.0]

In [ ]:
val_loss = []
val_acc = []

loss = 0
acc = 0
criterion = nn.CrossEntropyLoss()
attack = torchattacks.CW(model_ft, c=1e-4, kappa=0, steps=1000, lr=0.01)
loss, acc = test_model(model_ft, criterion, attack)
val_loss.append(loss)
val_acc.append(acc)




In [ ]:
val_loss = []
val_acc = []
for y in range(len(epsilon_vals)):
    loss = 0
    acc = 0
    criterion = nn.CrossEntropyLoss()
    attack = torchattacks.PGD(model_ft, eps=epsilon_vals[y], alpha=1/255, steps=40, random_start=True)
    loss, acc = test_model(model_ft, criterion, attack)
    val_loss.append(loss)
    val_acc.append(acc)




In [ ]:
import matplotlib.pyplot as plt
plt.plot(epsilon_vals, val_loss)

In [ ]:
import matplotlib.pyplot as plt
data = {'FGSM':0.6722338204592901, 'BIM':.467, 'PGD':0.373695, 'C & W':.02922}
courses = list(data.keys())
values = list(data.values())
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Adversarial Attack")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy for Different Adversarial Attacks on ResNet18")
plt.show()